In [ ]:
#start with importing the required libraries 
import openai as oi
import chromadb
from chromadb.utils import embedding_functions
import os, json 
import requests, time
from bs4 import BeautifulSoup

#init the global variables for further use 
OPENAI_API_KEY="<OPENAI_KEY>"
prefix = "https://www.ucl.ac.uk/museums-static/digitalegypt/alphabet.html"
stride = 400

In [ ]:
#init the client for ChromaDB
client = chromadb.PersistentClient(path="/Users/nermeen/Chroma")
client.heartbeat()

from chromadb.utils import embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                model_name="text-embedding-ada-002", 
                api_key = OPENAI_API_KEY
            )

In [ ]:
#create the collection in chromadb with the name ucl 
collection = client.create_collection(name = "ucl", embedding_function=openai_ef)

In [ ]:
#required imports
import requests
from bs4 import BeautifulSoup

currentURL = prefix + "alphabet.html"

#create the headers for a browser get request call 
headers = {
        'Connection': 'keep-alive',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36', 
        'Upgrade-Insecure-Requests': '1',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate'
}

#start the GET request to fetch the alphabet page, which has the appendix of items 
response = requests.get(currentURL, stream=True, headers = headers)

#parse the output of the GET/HTML using beautiful soup 
soup = BeautifulSoup(response.text, 'html.parser')

#find all the links in the file and start the counters
links = soup.find_all("a")
length = len(links)
currentID = 0
counter = 0 

#iterate over each link using the counter 
#add to the collection using the stride
while counter < length: 
    #re-init the list created 
    documents = []
    ids = [] 
    metadatas = []
    
    while(True):
        counter = counter + 1
        #skip the headers
        if(counter<6): 
            continue 
        #exit to add the lists so far to the collection and restart the loop when you reach the stride or list length
        if((counter%stride==0 and counter!=0) or counter==length):
            print("Counter is at "+str(stride)+"x: ", counter)
            break
        if(counter%100==0):
            print("Counter is at ", counter)
       
        #get the current link and reformat 
        link = links[counter]
        link = prefix + str(link.get("href"))
       
        if("#" in str(link) or "None" in str(link)):
            continue
        try:
            #start the request and parse the HTML 
            response1 = requests.get(link, stream=True, headers = headers)
            soup = BeautifulSoup(response1.text, 'html.parser')
            #find the title and split line by line 
            title = soup.title.string
            pagetext = str(soup.getText()).splitlines()
            
            counter2 = 0 
            document = ""
            #iterate over each line and truncate after 383 lines to avoid error with loading to collection 
            #ignore the intital lines 
            for item in pagetext: 
                if(not(item.strip()==None) and counter2>18 and not(item.strip()=="") and counter2<383):
                    document = document + str(item) + " "
                counter2 = counter2 + 1
            
            #inlcude the new record to all the lists 
            ids.append(str(currentID))
            documents.append(document)
            metadatas.append({'id': str(currentID)})
            currentID = currentID + 1 
        except Exception as e: 
            print(e)
    try: 
        #after hitting the stride, add the details to the collection 
        collection.add(documents = documents, metadatas = metadatas, ids = ids)
        print("data added to collection ", counter)
    except Exception as e: 
        #to handle throttling of OpenAI, rerun the request with defined pauses 
        if("Please try again" in str(e)): 
            print("Waiting 7m12s to start")
            time.sleep(432)
            collection.add(documents = documents, metadatas = metadatas, ids = ids)
        else: 
            skip = True
            print(counter)
            print(e)  

print("Counter after adding the collection ", counter)